# Script Testing

In [1]:
from selenium import webdriver
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

import win32com.client as wincl
# import os
from threading import Timer
import pandas as pd
import sys
import numpy as np
# from threading import Thread
import multiprocessing
from multiprocessing import SimpleQueue
import functools
from joblib import Parallel, delayed

echo = wincl.Dispatch("SAPI.SpVoice")
echo.rate = 4 #-1

chrome_option = Options()
chrome_option.add_argument('--headless')
chrome_option.add_argument('--disable-gpu')
chrome_option.add_argument('log-level = 3')
chrome_option.add_argument('window-size=1920x1080')

In [2]:
course_file = 'C:/Users/Tom/Desktop/tcl/test.csv'
df = pd.read_csv(course_file, dtype = {'num_course': pd.Int64Dtype()})
df = df.dropna(axis = 0, how = 'all').reset_index(drop=True)
df = df.fillna(np.nan)
df = df.drop_duplicates(subset = 'url', keep = 'first')

url = df.iloc[0].url
num_course = df.iloc[0].num_course
browser = webdriver.Chrome(executable_path=r"C:/Users/Tom/Desktop/tcl/chromedriver", chrome_options = chrome_option)
browser.get(url)
browser.maximize_window()
current_url_str = 'https://passportyork.yorku.ca/ppylogin/ppylogin'
current_url = browser.current_url 

<ipython-input-2-0a7e86becdbf>:9: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(executable_path=r"C:/Users/Tom/Desktop/tcl/chromedriver", chrome_options = chrome_option)


In [3]:
if current_url == current_url_str:
    name = browser.find_element_by_id("mli")
    name.send_keys("memorays")
    pwd = browser.find_element_by_id("password")
    pwd.send_keys("asd65656")
    element=WebDriverWait(browser,10).until(EC.element_to_be_clickable((By.NAME,"dologin")))
    browser.execute_script("arguments[0].click();", element)

In [4]:
num_course

3

In [5]:
ava_list = []
java_script = 'var items = {}; for (index = 0; index < arguments[0].attributes.length; ++index) { items[arguments[0].attributes[index].name] = arguments[0].attributes[index].value }; return items;'
        
#for idx_course in range(num_course): 
idx_course = 0
course_xpath = "//div[@class = 'courseDiv bc{} bd{}']".format(idx_course + 1, idx_course + 1)
requirement = WebDriverWait(browser, 60).until(
    EC.presence_of_element_located((By.XPATH, course_xpath)))
myElem = browser.find_element_by_xpath(course_xpath)
check_box = myElem.find_element_by_xpath("div[@class = 'dropDownText']").find_element_by_xpath("span[@class = 'class_checkboxes']")
labels = check_box.find_elements_by_css_selector("label")
course_name = myElem.text.split('\n')[1].split(' ')[0]
course_name

'HH-PSYC-4030-6.00-EN'

In [6]:
myElem.text

'Keele  Select...   \nHH-PSYC-4030-6.00-EN   \nAll classes are full\nHH-PSYC-4180-6.00-EN has no classes with open seats that do not conflict with this course\nAll classes of this course conflict with all classes of HH-PSYC-4220-6.00-EN\nBehaviour Modification and Behaviour Therapy\nAll Terms (1)\nTerm SU\nAll Sections (1)\nSection A\n    Try all classes (1)\n    \n    Try specific classes...\nSection A: Term SU: Semr 01 (Full)\n    Classes:\nTerm SU: Section A: Semr 01 (Full)  \nDescription: An examination of theoretical issues, basic research and practical application in the area of behaviour change. Prerequisites: HH/PSYC 1010 6.00, with a minimum grade of C; HH/PSYC 2030 3.00; one of HH/PSYC 2021 3.00, HHPSYC 2020 6.00; HH/PSYC 2130 3.00;HH/PSYC 3140 3.00\n\n ...Show More '

In [8]:
myElem.text.split('\n')[2]

'All classes are full'

In [8]:
labels

[<selenium.webdriver.remote.webelement.WebElement (session="74c5fab0c6c75c35e2ef86fd39758336", element="8cec4423-3ae6-4bda-a9fe-0b12ccddf29a")>]

In [9]:
for i in range(len(labels)):
    label = labels[i]
    idx = browser.execute_script(java_script, label)['for']
    input_val = label.find_element_by_xpath("//input[@id = '{}']".format(idx))
    attrs = browser.execute_script(java_script, input_val)

In [13]:
attrs

{'checked': '',
 'class': 'class_chk',
 'data-cfsi': '0',
 'id': '0-03711766299311692',
 'type': 'checkbox'}

In [14]:
label.text

'Term SU: Section A: Semr 01 (Full)  '

In [23]:
if 'checked' in attrs.keys():
    course_details = label.text
    if course_details:
        if '(Full)' not in course_details:
            full_course_name = course_name[3:12] + ' ' + course_details[5:7] + ' ' + course_details[16:18] + course_details[-5:-2]
            remind_name = course_name[3:12]
            echo.Speak(remind_name)
            print(full_course_name)
            ava_list.append(full_course_name)

In [24]:
full_course_name

'PSYC-4030 SU  All)'

In [25]:
course_details

'Term SU: Section A: Semr 01 (Full)  '

In [26]:
course_details[-5:-2]

'll)'

In [2]:
import sqlite3
import sys
from os import getenv, path
import os
from Crypto.Cipher import AES
from Crypto.Protocol.KDF import PBKDF2
import keyring

def get_cookies(url, cookiesfile):

    def chrome_decrypt(encrypted_value, key=None):
        dec = AES.new(key, AES.MODE_CBC, IV=iv).decrypt(encrypted_value[3:])
        decrypted = dec[:-dec[-1]].decode('utf8')
        return decrypted

    cookies = []
    if sys.platform == 'win32':
        import win32crypt
        conn = sqlite3.connect(cookiesfile)
        cursor = conn.cursor()
        cursor.execute(
            'SELECT name, value, encrypted_value FROM cookies WHERE host_key == "' + url + '"')
        for name, value, encrypted_value in cursor.fetchall():
            if value or (encrypted_value[:3] == b'v10'):
                cookies.append((name, value))
            else:
                decrypted_value = win32crypt.CryptUnprotectData(
                    encrypted_value, None, None, None, 0)[1].decode('utf-8') or 'ERROR'
                cookies.append((name, decrypted_value))

    elif sys.platform == 'linux':
        my_pass = 'peanuts'.encode('utf8')
        iterations = 1
        key = PBKDF2(my_pass, salt, length, iterations)
        conn = sqlite3.connect(cookiesfile)
        cursor = conn.cursor()
        cursor.execute(
            'SELECT name, value, encrypted_value FROM cookies WHERE host_key == "' + url + '"')
        for name, value, encrypted_value in cursor.fetchall():
            decrypted_tuple = (name, chrome_decrypt(encrypted_value, key=key))
            cookies.append(decrypted_tuple)
    else:
        print('This tool is only supported by linux and Mac')

    conn.close()
    return cookies


if __name__ == '__main__':
    pass
else:
    salt = b'saltysalt'
    iv = b' ' * 16
    length = 16

#get_cookies('YOUR URL FROM THE COOKIES', 'YOUR PATH TO THE "/Default/Cookies" DATA')
url = 'https://wrem.sis.yorku.ca/Apps/WebObjects/REM.woa/wa/DirectAction/rem'
path = 'C:/Users/Tom/AppData/Local/Google/Chrome/User Data/Default/Cookies'
get_cookies(url, path)

[]

In [3]:
import browser_cookie3
cookies = browser_cookie3.chrome(domain_name='.google.com')
response = requests.get('http://www.google.com', verify=False, headers=headers, cookies=cookies, timeout=3)

ModuleNotFoundError: No module named 'Crypto'